**Multimodel conversation**

In [6]:
import os 
from dotenv import load_dotenv
from openai import OpenAI
import anthropic

In [7]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
claude_api_key = os.getenv("ANTHROPIC_API_KEY")

In [8]:
gpt_system_prompt = "You are in a debating competition and you are the opposition of any given topic. \
You will talk in turns with the affirmative side and each time you will have to put a statement of 50 words"

claude_system_prompt = "You are in a debating competition and you are on the affirmative side. \
You will talk in turns with the opposition side and each time you will have to put a statement of 50 words"

topic = "The topic is - nuclear power is the key to green energy"

gpt_messages = [""]
claude_messages = [""]
google_messages = [""]

In [9]:
# function to send and get the response form gpt 

openai = OpenAI()

def call_gpt():
    messages = [{"role": "system", "content": gpt_system_prompt},
               {"role": "user","content": topic}]

    
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    
    completion = openai.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=100,
        temperature=0.7
    )
    
    return completion.choices[0].message.content

call_gpt()

'While nuclear power is low in carbon emissions, it presents significant challenges such as radioactive waste management, high costs, and potential for catastrophic accidents. Renewable energy sources like solar and wind, which are increasingly cost-effective and safer, should be prioritized for a sustainable and green energy future.'

In [11]:
# function to send and get the response form claude

claude = anthropic.Anthropic()

def call_claude():
    

    message = claude.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=100,
        temperature=1,
        system=claude_system_prompt,
        messages=[{"role": "user", "content": topic}]
    )
    
    return message.content[0].text

call_claude()

'**First Affirmative (50 words)**\n\nNuclear power is our most viable path to a green energy future. It produces zero carbon emissions during operation, has the highest energy density, and provides reliable baseload power regardless of weather conditions. Unlike solar and wind, nuclear plants require minimal land use while delivering maximum energy output, making it environmentally superior.'

In [ ]:
# function to display the conversation one by one 
